In [1]:
# Importing all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import json

In [2]:
# each records collecting to the list
records = []
with open('C:\\Users\\manik\\Downloads\\kindle_reviews\\kindle_reviews.json', 'r') as file:
    for line in file:
        json_object = json.loads(line)
        records.append(json_object)

In [3]:
import psycopg2

In [4]:
connection = psycopg2.connect(host='localhost',
                             user='postgres',
                             port=5432,
                             database='postgres',
                             password='ManiAnu@12345')
connection.set_session(readonly=False, autocommit=True)
cur = connection.cursor()

In [5]:
def sanitize(data):
    """Recursively remove NUL characters from all strings in a data structure."""
    if isinstance(data, str):
        return data.replace('\0', '')
    elif isinstance(data, dict):
        return {key: sanitize(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [sanitize(item) for item in data]
    return data

In [6]:
insert_query = """
INSERT INTO reviews (reviewerID, asin, reviewText, summary, helpful)
VALUES (%s, %s, %s, %s, %s);
"""

In [7]:
review_records = [(item['reviewerID'], item['asin'], item['reviewText'], item['summary'],  item['helpful']) for item in records]

In [8]:
sanitized_records = [sanitize(item) for item in review_records]

In [9]:
len(records)

982619

In [10]:
def sanitize_string(input_string):

    sanitized = input_string.replace('\x00', '')

    return sanitized


In [11]:
def sanitize_record(record):
    # Assuming record is a dictionary with multiple text fields
    for key, value in record.items():
        if isinstance(value, str):
            record[key] = sanitize_string(value)
    return record


In [12]:
sanitized_records = [sanitize_record(record) for record in records]

In [13]:
records_to_be_inserted = [(item['reviewerID'], item['asin'], item['reviewText'], item['summary'], item['helpful']) for item in sanitized_records]

In [ ]:
cur.executemany(insert_query, records_to_be_inserted)
connection.commit()